In [75]:
import numpy as np
import matplotlib.pyplot as plt

# Sudoku Solver using Annealing

Napisz program poszukujący rozwiązania łamigłówki Sudoku za pomocą symulowanego
wyżarzania. Plansza 9 ×9 ma zostać wczytana z pliku tekstowego, w którym pola puste
zaznaczone są znakiem x. Jako funkcję kosztu przyjmij sumę powtórzeń cyfr występu-
jących w wierszach bloku 9 ×9, kolumnach bloku 9 ×9 oraz blokach 3 ×3. Zaproponuj
metodę generacji stanu sąsiedniego. Przedstaw zależność liczby iteracji algorytmu od
liczby pustych miejsc na planszy. Czy Twój program jest w stanie znaleźć poprawne
rozwiązanie dla każdej z testowanych konfiguracji wejściowych?

## Table of Contents

1. [Read Sudokus](#1-read-sudokus)
2. [Fill Sudoku](#2-fill-sudoku)
3. [Annealing](#3-annealing)  
    3.1 [Cost Function](#31-cost-function)  
    3.2 [Annealing Algorithm](#32-annealing-algorithm)
4. [Vizualization](#4-vizualization)
5. [Results](#5-results)

## 1. Read Sudokus

sudokus were taken from [http://magictour.free.fr/top1465](http://magictour.free.fr/top1465)

In [76]:
def read_sudokus(filename="sudoku.txt"):
    with open(filename, "r") as f:
        sudokus = f.readlines()
    sudokus = [list(sudoku.strip()) for sudoku in sudokus]
    return np.array([np.reshape(row, (9, 9)) for row in sudokus])

## 2. Fill Sudoku

In [77]:
def fill_sudoku(sudoku):
    positions = np.argwhere(sudoku == "x")
    np.random.shuffle(positions)
    numbers, counts = np.unique(sudoku, return_counts=True)
    idx = 0
    ranger = 0
    for num in [str(i) for i in range(1, 10)]:
        if num not in numbers:
            ranger = 9
        else:
            ranger = 9 - counts[np.where(numbers == num)][0]
        for _ in range(ranger):
            sudoku[positions[idx][0], positions[idx][1]] = num
            idx += 1
    return sudoku

## 3. Annealing

### 3.1 Cost Function

In [78]:
def count_inital_energy(sudoku):
    energy = 0
    for i in range(9):
        _, cnt = np.unique(sudoku[i], return_counts=True)
        energy += np.sum(cnt-1)
        _, cnt = np.unique(sudoku[:, i], return_counts=True)
        energy += np.sum(cnt-1)
        _, cnt = np.unique(sudoku[i//3*3:i//3*3+3, i%3*3:i%3*3+3], return_counts=True)
        energy += np.sum(cnt-1)
    return energy

In [79]:
def count_swap_energy(sudoku, x1, y1, x2, y2):
    current = 0
    _, cnt = np.unique(sudoku[x1], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[x2], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[:, y1], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[:, y2], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[x1//3*3:x1//3*3+3, y1//3*3:y1//3*3+3], return_counts=True)
    current += np.sum(cnt-1)
    _, cnt = np.unique(sudoku[x2//3*3:x2//3*3+3, y2//3*3:y2//3*3+3], return_counts=True)
    current += np.sum(cnt-1)
    return current

### 3.2 Annealing Algorithm

In [94]:
def annealing(sudoku, temp=1, step=0.999, acceptance=0.5, max_iter=100000):
    sudoku = fill_sudoku(sudoku)
    energy = count_inital_energy(sudoku)
    data = {'iter': [], 'temp':[], 'energy': []}
    notchanged = 0

    for i in range(max_iter):
        if energy == 0:   sudoku, data, True
        if notchanged > 1000: temp = min(1, temp+0.3)
        x1, y1, x2, y2 = np.random.randint(0, 9, 4)
        prev = count_swap_energy(sudoku, x1, y1, x2, y2)
        sudoku[x1, y1], sudoku[x2, y2] = sudoku[x2, y2], sudoku[x1, y1]
        current = count_swap_energy(sudoku, x1, y1, x2, y2)
        if current < prev or (current - prev) * np.random.rand() < acceptance * temp:
            energy += current - prev
            notchanged = 0
        else:
            sudoku[x1, y1], sudoku[x2, y2] = sudoku[x2, y2], sudoku[x1, y1]
            notchanged += 1
        temp *= step

        data['iter'].append(i)
        data['temp'].append(temp)
        data['energy'].append(energy)

    return sudoku, data, False

## 4. Vizualization

In [81]:
def plot(data):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].plot(data['iter'], data['temp'])
    ax[0].set_title('Temperature')
    ax[1].plot(data['iter'], data['energy'])
    ax[1].set_title('Energy')
    plt.show()

## 5. Results

In [95]:
sudokus = read_sudokus()
#s1 = sudokus[0]
s1 = np.array([['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
               ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
                ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
                ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
                ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
                ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
                ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
                ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'],
                ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']])
#sudoku, data, _ = annealing(s1)
#plot(data)
ixds = 0
for s in sudokus:
    _, data, result = annealing(s)
    if result and ixds < 5:
        ixds += 1
        plot(data)
print(ixds)

KeyboardInterrupt: 

---